<a href="https://colab.research.google.com/github/ctix/chinese-stock-api/blob/master/simplify_class_and_test_fun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The new way to make the structure 
---------
Study for modification  \\
Copy from cstock directory base_engine.py 

In [0]:
import abc

class Engine(object):
    """Base Engine class, must be inherited
    """

    __slots__ = ['_url']

    def __init__(self, base_url=None):
        if base_url is None:
            self._url = self.DEFAULT_BASE_URL
        else:
            self._url = base_url
 
    @abc.abstractmethod
    def parse(self, data, stock_id):
        """
        parse the data from service URL
        :param data:             payload string
        :type data:            ``str``
        :param stock_id:         raw stock/fund id
        :type stock_id:        ``str``
        :returns:            tuple of Stock objects
        :rtype:            ``tuple``
        """
        pass

    def get_url(self, stock_id, date=None):
        """
        transform stock_id date into service URL
        :param stock_id:            stock id  
        :type stock_id:            ``str``
        :param date:            tuple of start date & end date, optional
            e.g. ('2014-12-13', '2014-12-15')
        :type date:            ``tuple``
        :returns:            service URL
        :rtype:            ``str``
        """
        engine_id = self.get_engine_id(stock_id)
        return self._url % engine_id

    def get_engine_id(self, stock_id):
        """
        transform raw stock_id into service stock id
        we regard stock/fund starting with 0 or 3 belongs to shenzhen
        :param stock_id:            raw stock id
        :type stock_id:            ``str``
        :returns:            service stock id
        :rtype:            ``str``
        """
        if stock_id.startswith('0') or stock_id.startswith('3'):
            return self.shenzhen_transform(stock_id)
        
        if stock_id.startswith('6'):
            return self.shanghai_transform(stock_id)
        
        raise ParserException("Unknow stock id %s" % stock_id)

## Copy from the cstock/model.py

In [0]:
class ParserException(Exception):
    pass


class Stock(object):
    
    # yesterday_close is yesterday close price
    # close is today close price

    # volume: unit of stock transacted
    # turnover: total transaction money

    __slots__ = [
        'name',
        'code',
        'date',
        'time',
        'price',
        'open',
        'close',
        'high',
        'low',
        'volume',
        'turnover',
        'yesterday_close',
    ]

    def __init__(self, **argvs):
        
        for (k, v) in argvs.items():
            setattr(self, k, v)

    def as_dict(self):
        result = {
            i: getattr(self, i, None)
            for i in self.__slots__
        }

        # dispose date and time because they are datetime class instance
        if result['date'] is not None:
            result['date'] = str(result['date'])

        if result['time'] is not None:
            result['time'] = str(result['time'])

        return result

13.80738004562425
18.19993988938782
0.9728675345252052


In [0]:
# ##chinese-stock-api/cstock/request.py

import urllib2


class Requester(object):
    """ Requester class for diferent engine
    """

    __slots__ = ["_engine"]

    def __init__(self, engine):

        self._engine = engine

    def request(self, stock_id, date=None):
        """request by stock id and date
        :param stock_id:
            stock id string
        :type stock_id:
            ``str``
        :param date:
            tuple of start date and stop date
            e.g. ('2014-03-04', '2014-03-05')
        :type date:
            ``tuple``
        :returns:
            tuple of stock objects
        :rtype:
            ``tuple``
        """
        
        stock_url = self._engine.get_url(stock_id, date)
        print stock_url

        request = urllib2.Request(stock_url)
        request.add_header('Content-Type', 'application/json')
        response = urllib2.urlopen(request)
        data = response.read()

        return self._engine.parse(data, stock_id)

        0
0   Geeks
1     For
2   Geeks
3      is
4  portal
5     for
6   Geeks


    Name  Age
0    Tom   20
1   nick   21
2  krish   19
3   jack   18


In [0]:
# Import pandas package
import pandas as pd
 
# Define a dictionary containing employee data
data = {'Name':['Jai', 'Princi', 'Gaurav', 'Anuj'],
        'Age':[27, 24, 22, 32],
        'Address':['Delhi', 'Kanpur', 'Allahabad', 'Kannauj'],
        'Qualification':['Msc', 'MA', 'MCA', 'Phd']}
 
# Convert the dictionary into DataFrame 
df = pd.DataFrame(data)
 
# select two columns
print(df[['Name', 'Qualification']])

     Name Qualification
0     Jai           Msc
1  Princi            MA
2  Gaurav           MCA
3    Anuj           Phd


In [0]:
# importing pandas as pd
import pandas as pd
 
# importing numpy as np
import numpy as np
 
# dictionary of lists
dict = {'First Score':[100, 90, np.nan, 95],
        'Second Score': [30, 45, 56, np.nan],
        'Third Score':[np.nan, 40, 80, 98]}
 
# creating a dataframe from dictionary
df = pd.DataFrame(dict)
 
# filling missing value using fillna()  
df.fillna(0)

In [0]:
df

In [0]:
# importing pandas as pd
import pandas as pd
 
# importing numpy as np
import numpy as np
 
# dictionary of lists
dict = {'First Score':[100, 90, np.nan, 95],
        'Second Score': [30, 45, 56, np.nan],
        'Third Score':[np.nan, 40, 80, 98]}
 
# creating a dataframe from dictionary
df = pd.DataFrame(dict)
 
# filling missing value using fillna()  
df.fillna(0)

,First Score,Second Score,Third Score
0,100.0,30.0,0.0
1,90.0,45.0,40.0
2,0.0,56.0,80.0
3,95.0,0.0,98.0


In [0]:
# importing pandas as pd
import pandas as pd
   
# dictionary of lists
dict = {'name':["aparna", "pankaj", "sudhir", "Geeku"],
        'degree': ["MBA", "BCA", "M.Tech", "MBA"],
        'score':[90, 40, 80, 98]}
  
# creating a dataframe from a dictionary 
df = pd.DataFrame(dict)
 
print(df)

     name  degree  score
0  aparna     MBA     90
1  pankaj     BCA     40
2  sudhir  M.Tech     80
3   Geeku     MBA     98


In [0]:
# creating a list of dataframe columns
columns = list(df)
 
for i in columns:
 
    # printing the third element of the column
    print (df[i][2])

sudhir
M.Tech
80
